In [83]:
from qiskit.circuit import QuantumCircuit, Parameter
import numpy as np
phi = Parameter("1")
phi2 = Parameter("2")
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.rx(phi,qubit=[0])
circuit.rx(phi2,qubit=[1])
print(circuit)
print(circuit.parameters)
value = np.random.random()
parameters = [value for _ in range(2)]
print(circuit.bind_parameters(parameters))

       ┌───┐┌───────┐
q68_0: ┤ H ├┤ Rx(1) ├
       ├───┤├───────┤
q68_1: ┤ H ├┤ Rx(2) ├
       ├───┤└───────┘
q68_2: ┤ H ├─────────
       ├───┤         
q68_3: ┤ H ├─────────
       └───┘         
ParameterView([Parameter(1), Parameter(2)])
       ┌───┐┌───────────────────────┐
q68_0: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤├───────────────────────┤
q68_1: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤└───────────────────────┘
q68_2: ┤ H ├─────────────────────────
       ├───┤                         
q68_3: ┤ H ├─────────────────────────
       └───┘                         


In [23]:
individual = "0000000"
elitism_population = ["1110111","1111111"]
match_percentage = []
def match(subject_1,subject_2):
    return sum([int(subject_1[i] == subject_2[i]) for i in range(0,len(subject_1))])/len(subject_1)
for elite in elitism_population:
    match_percentage.append(match(individual,elite))
    print(match_percentage)
print(sum(match_percentage)/len(match_percentage))

[0.14285714285714285]
[0.14285714285714285, 0.0]
0.07142857142857142


In [11]:
from Experiments import saveLoad
from CLQGA import Individual
output = saveLoad("load","experiment_10_april_ideal_1024_0", None)
experiment_population = output[0]
experiment_duration = output[1]
experiment_average_fitness_50_increment = output[2]
experiment_average_crowd_score_50_increment = output[3]

In [12]:
# [population,time.gmtime((time.perf_counter() - start_total_run_time)),data_average_fitness, data_average_crowd_score, data_average_error_score]
print(output)

[[<CLQGA.Individual object at 0x7fcc96c7b950>, <CLQGA.Individual object at 0x7fcc96a8d650>, <CLQGA.Individual object at 0x7fcc96aa3510>, <CLQGA.Individual object at 0x7fcc96aa1c90>, <CLQGA.Individual object at 0x7fcc96aa32d0>, <CLQGA.Individual object at 0x7fcc96aa04d0>, <CLQGA.Individual object at 0x7fcc96aa3410>, <CLQGA.Individual object at 0x7fcc96aa3190>, <CLQGA.Individual object at 0x7fcc96aa3250>, <CLQGA.Individual object at 0x7fcc96aa11d0>, <CLQGA.Individual object at 0x7fcc96aa0a90>, <CLQGA.Individual object at 0x7fcc96aa3d50>, <CLQGA.Individual object at 0x7fcc96aa1490>, <CLQGA.Individual object at 0x7fcc96aa0450>, <CLQGA.Individual object at 0x7fcc96aa1ed0>, <CLQGA.Individual object at 0x7fcc96aa2050>, <CLQGA.Individual object at 0x7fcc96aa2710>, <CLQGA.Individual object at 0x7fcc96aa1e90>, <CLQGA.Individual object at 0x7fcc96aa0510>, <CLQGA.Individual object at 0x7fcc96aa2ad0>, <CLQGA.Individual object at 0x7fcc96aa2090>, <CLQGA.Individual object at 0x7fcc96aa0390>, <CLQGA.I